In [ ]:
#This code needs a high memory machine to run

In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import time
import os
from pyprojroot import here
import resource

In [6]:
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"
#base_data_folder = "/home/leon/scratch/custom/"

In [ ]:
#from braincelldata.org
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad")

In [ ]:
adata_macosko.obs['cell.type'] = adata_macosko.obs['ClusterNm'].astype(str)
adata_macosko.obs['study_id'] = "Macosko"
adata_macosko.obs.index = adata_macosko.obs.index.to_numpy(dtype="str")

In [ ]:
print("Number of Macosko cell types: " + str(len(set(adata_macosko.obs["cell.type"]))))


In [ ]:
##########
### Zeng
##########

In [ ]:
adata_Zeng = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad")

In [ ]:
adata_Zeng.obs.groupby('method').count()

In [ ]:
base_data_folder + "whole_mouse_brain/zeng/from_aws/"

In [ ]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')

In [ ]:
#set study ID here to prevent view as actual memory grabs 
Zeng_cluster_info_AWS['study_id'] = "Zeng"
Zeng_cluster_info_AWS['cell.type'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [ ]:
#set it to be strings
adata_Zeng.obs['cl'] = adata_Zeng.obs['cl'].astype(str)
Zeng_cluster_info_AWS['cl'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [ ]:
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs = adata_Zeng.obs.merge(Zeng_cluster_info_AWS, how="left", on='cl')

In [ ]:
#use ensmbl ID as index
adata_Zeng.var = adata_Zeng.var.set_index("gene_identifier")

In [ ]:
#cells lost
adata_Zeng.obs['cluster_id'].isna().sum() #none it seems

In [ ]:
#LQ or low quality cells
sum(adata_Zeng.obs['subclass_label'] == "LQ")

In [ ]:
#remove #LQ 
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs["filter_out"] = adata_Zeng.obs['subclass_label'] == "LQ"
adata_Zeng = adata_Zeng[~adata_Zeng.obs["filter_out"],:]
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)

In [ ]:
print("Number of Zeng cell types: " + str(len(set(adata_Zeng.obs["cell.type"]))))

In [ ]:
merged=ad.concat([adata_Zeng, adata_macosko], join="inner")

In [ ]:
#free memory
adata_Zeng = None
adata_macosko = None

In [ ]:
merged.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.h5ad")

In [ ]:
#write out subsets for HVGs

In [ ]:
frac = 0.10
rng = np.random.default_rng(42)
n = merged.n_obs
k = max(1, int(n * frac))        # number of cells to keep
idx = np.sort(rng.choice(n, size=k, replace=False))

adata_sub = merged[idx].copy()    # materialize subset in memory
adata_sub.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.10pct.h5ad")

In [ ]:
frac = 0.010
rng = np.random.default_rng(42)
n = merged.n_obs
k = max(1, int(n * frac))        # number of cells to keep
idx = np.sort(rng.choice(n, size=k, replace=False))

adata_sub = merged[idx].copy()    # materialize subset in memory
adata_sub.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.01pct.h5ad")

In [ ]:
#merged = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.01pct.h5ad")

In [ ]:
#merged = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.10pct.h5ad")

In [ ]:
merged = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.h5ad")

In [ ]:
#write out merged? memory outage?
###Seurat HVG

In [ ]:
#3534 was the # from metaneighbor

In [ ]:
merged.var.shape[0]

In [ ]:
sc.pp.highly_variable_genes(merged, batch_key = "study_id", flavor = "seurat_v3_paper", n_top_genes=3534, inplace=True)

In [ ]:
mem_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
# print the memory usage in megabytes
print("Peak memory use in Gb  " + str(round(mem_usage / 1024 / 1024,2 )) + " PID  " + str(os.getpid()))


In [ ]:
#was 49Gb and 452

In [ ]:
merged.var.to_csv(base_data_folder + "/whole_mouse_brain/processed/merged_var_after_seurat_HVG_3534.csv")

In [ ]:
sc.pp.highly_variable_genes(merged, batch_key = "study_id", flavor = "seurat_v3_paper", n_top_genes=merged.var.shape[0], inplace=True)

In [ ]:
merged.var.to_csv(base_data_folder + "/whole_mouse_brain/processed/merged_var_after_seurat_HVG_all.csv")

In [ ]:
sc.pp.highly_variable_genes(merged, batch_key = "study_id", flavor = "seurat_v3_paper", n_top_genes=1767, inplace=True)

In [ ]:
merged.var.to_csv(base_data_folder + "/whole_mouse_brain/processed/merged_var_after_seurat_HVG_1767.csv")

In [ ]:
#merged.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_after_seurat_v3_HVG.h5ad")

In [ ]:
sc.pp.highly_variable_genes(merged, batch_key = "study_id", flavor = "seurat_v3_paper", n_top_genes=3534, inplace=True)

In [ ]:
#subset and write out

In [ ]:
merged.var.highly_variable.sum()

In [ ]:
merged = merged[:, merged.var['highly_variable']]

In [ ]:
merged

In [ ]:
merged.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_3534_Seurat_V3_HVG.h5ad")

In [ ]:
#quick code to create a 5% version for mixHVG R loading

In [4]:
temp = sc.read_h5ad("/vault/lfrench/mouse_brain_cluster_replicability/data/whole_mouse_brain/processed/merged_before_HVG.10pct.h5ad")

In [6]:
frac = 0.5
rng = np.random.default_rng(42)
n = temp.n_obs
k = max(1, int(n * frac))        # number of cells to keep
idx = np.sort(rng.choice(n, size=k, replace=False))

adata_sub = temp[idx].copy()    # materialize subset in memory
adata_sub.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.05pct.h5ad")

In [7]:
adata_sub

AnnData object with n_obs × n_vars = 422513 × 21649
    obs: 'study_id', 'cell.type'

In [ ]:
#### subset to the R based mixhvg top genes from HVG_testing.R

In [7]:
merged = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_before_HVG.h5ad")

In [9]:
mixhvg = pd.read_csv(base_data_folder + "../results/HVG_lists/mixhvg_FindVariableFeaturesMix_3534_5percent.csv")

In [11]:
mixhvg = mixhvg['gene_id'].tolist()

In [13]:
merged.var

""
ENSMUSG00000051951
ENSMUSG00000025900
ENSMUSG00000025902
ENSMUSG00000033845
ENSMUSG00000025903
...
ENSMUSG00000095523
ENSMUSG00000095475
ENSMUSG00000094855
ENSMUSG00000095019


In [17]:
merged.var.index.isin(mixhvg)

array([ True, False,  True, ..., False, False,  True])

In [18]:
merged = merged[:, mixhvg]

In [19]:
merged

View of AnnData object with n_obs × n_vars = 8450272 × 3534
    obs: 'study_id', 'cell.type'

In [ ]:
merged.var['highly_variable'] = True

In [20]:
merged.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/merged_3534_mixhvg_HVG.h5ad")

In [21]:
base_data_folder + "/whole_mouse_brain/processed/merged_3534_mixhvg_HVG.h5ad"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/processed/merged_3534_mixhvg_HVG.h5ad'

In [22]:
print("Done")

Done
